In [1]:
import argparse
import os
import pickle
import sys
from collections import Counter
from datetime import datetime

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from tensorflow import keras
# from tensorflow.keras.layers import (Activation, Conv2D, Dense, Dropout, Flatten,
#                           MaxPooling2D)
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.utils import Sequence
from CNN_singleImage_baseline import DataGenerator

In [2]:
print(tf.__version__)
print("GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
GPUs Available:  1


In [3]:
# load the model
modelpath = '/data/models/'
os.listdir(modelpath)

['simpleCNN_1000epochs_20191112_024957.h5',
 'train.pickle',
 'simpleCNN_1000epochs_20191110_153647.h5',
 'simpleCNN_1000epochs_20191110_171702.h5',
 'test.pickle',
 'simpleCNN_history_10epochs_20191109_192747.pickle',
 'simpleCNN_10epochs_20191109_192747.h5',
 'simpleCNN_history_1000epochs_20191110_150328.pickle',
 'simpleCNN_100epochs_20191110_033120.h5',
 'simpleCNN_history_1000epochs_20191110_153647.pickle',
 'simpleCNN_history_1000epochs_20191110_171702.pickle',
 'simpleCNN_history_100epochs_20191110_033120.pickle',
 'simpleCNN_1000epochs_20191110_150328.h5',
 'predictions.pickle',
 'simpleCNN_history_1000epochs_20191112_024957.pickle']

In [4]:
modelfile = 'simpleCNN_1000epochs_20191112_024957.h5'
model = keras.models.load_model(os.path.join(modelpath, modelfile))

In [5]:
with open(os.path.join(modelpath, 'test.pickle'), 'rb') as f:
    x_test, y_test = pickle.load(f)
print(len(x_test), len(y_test))

524 524


In [6]:
batch_size = 1
test_generator = DataGenerator(x_test, y_test, batch_size)

In [7]:
y_pred = model.predict_generator(test_generator)

In [8]:
y_pred.shape

(524, 5)

In [9]:
np.sum(y_pred > 0.5, axis=0)

array([479,  26,   0,   0,   1])

In [10]:
import copy
y_prob = copy.copy(y_pred)
y_pred = (y_pred > 0.5).astype(np.int)
with open(os.path.join(modelpath, 'predictions.pickle'), 'wb') as f:
    pickle.dump((y_prob, y_pred), f, protocol=-1)

In [11]:
labelPath = 'week11_all_assigned.csv'
df = pd.read_csv(labelPath)

In [12]:
new_files = []
for file in df.clip_title:
    newfile = ''.join(file.split('.mp4')) + '.mp4'
    new_files.append(newfile)
df['clip_title'] = new_files
df['label'] = df['class']
df.drop(columns=['class'], inplace=True)
df.groupby('label').size()

label
-1.0       53
 0.0     1892
 1.0      497
 2.0       22
 3.0      132
 4.0       76
 99.0    2769
dtype: int64

In [13]:
df = df.loc[df.label != -1]
df = df.loc[df.label != 99]
df.groupby('label').size()

label
0.0    1892
1.0     497
2.0      22
3.0     132
4.0      76
dtype: int64

In [14]:
df.shape

(2619, 4)

In [15]:
# vidPath = '/data/vids/scaled'
filenames = [f.split('.mp4')[0] + '_scaled.mp4' for f in df.clip_title]
# filenames = [os.path.join(vidPath, f) for f in filenames]
# labels = df.punch.tolist()
labels = df.label.tolist()


In [16]:
df['clip_title'] = filenames

In [17]:
df2 = pd.DataFrame()

In [18]:
filenames2 = [x.split('/')[-1] for x in x_test]
df2['clip_title'] = filenames2

In [24]:
df2 = df2.merge(df, how='inner', on='clip_title')
df2

,clip_title,video-UID,labeler,label
0,downward_strike_punches_slice655_scaled.mp4,2857,Alex,0.0
1,downward_strike_punches_slice275_scaled.mp4,12907,Jen,0.0
2,traditional_dance_slice415_scaled.mp4,5205,Jen,0.0
3,V_789_slice325_scaled.mp4,12430,Ahsen,1.0
4,downward_strike_punches_slice850_scaled.mp4,13035,Lance,0.0
...,...,...,...,...
519,streetfight1_slice615_scaled.mp4,4441,Lance,0.0
520,streetfight4_slice410_scaled.mp4,15008,Jen,0.0
521,V_991_scaled.mp4,12677,Lance,0.0
522,how_to_strike_max_power_slice180_scaled.mp4,13724,Lance,0.0


In [25]:
from sklearn.preprocessing import OneHotEncoder

In [26]:
ohe = OneHotEncoder()
ohe_labels = ohe.fit_transform(df2['label'].values.reshape(-1,1))
ohe_labels.shape

/root/w210/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(524, 5)

In [27]:
oneHots = []
ohe_labels = ohe_labels.toarray()
for i in range(ohe_labels.shape[0]):
    oneHots.append(ohe_labels[i])
len(oneHots), len(oneHots[0])

(524, 5)

In [28]:
df2['ohe_labels'] = oneHots

In [30]:
probs = []
preds = []
for i in range(y_prob.shape[0]):
    probs.append(y_prob[i])
    preds.append(y_pred[i])
len(probs), len(probs[0]), len(preds), len(preds[0])

(524, 5, 524, 5)

In [32]:
df2['y_prob'] = probs
df2['y_pred'] = preds

In [31]:
# order is maintained
for i,f in enumerate(filenames2):
    if f != df2.clip_title[i]:
        print(i,f)

In [33]:
# with open(os.path.join(modelpath, 'predictions.pickle'), 'wb') as f:
#     pickle.dump((y_prob, y_pred), f, protocol=-1)
df2.to_pickle(os.path.join(modelpath, 'predictions.pickle'))

In [34]:
df = pd.read_pickle('/data/models/predictions.pickle')
df.head()

,clip_title,video-UID,labeler,label,ohe_labels,y_prob,y_pred
0,downward_strike_punches_slice655_scaled.mp4,2857,Alex,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0]","[0.91471887, 0.04957079, 3.3734228e-05, 0.0336...","[1, 0, 0, 0, 0]"
1,downward_strike_punches_slice275_scaled.mp4,12907,Jen,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0]","[0.7162727, 0.13020208, 0.0011639277, 0.133737...","[1, 0, 0, 0, 0]"
2,traditional_dance_slice415_scaled.mp4,5205,Jen,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0]","[0.76954263, 0.19615111, 0.008012304, 0.012842...","[1, 0, 0, 0, 0]"
3,V_789_slice325_scaled.mp4,12430,Ahsen,1.0,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.35030097, 0.64757156, 0.0005246858, 0.00062...","[0, 1, 0, 0, 0]"
4,downward_strike_punches_slice850_scaled.mp4,13035,Lance,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0]","[0.73543257, 0.07232628, 1.3397551e-05, 0.1898...","[1, 0, 0, 0, 0]"
